In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv
/kaggle/input/sample/scraped_tweets.csv


In [2]:
!pip install transformers

In [35]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

# Torch ML libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

# Misc.
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm,trange,tqdm_notebook

In [2]:
import nltk
# nltk.download("movie_reviews")
import numpy as np
from nltk.corpus import movie_reviews 
import pandas as pd
from nltk.corpus import twitter_samples 
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop = stopwords.words('english')
import string
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix

from sklearn.ensemble import StackingClassifier, VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import pickle

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

## Helper Methods for Cleaning Data 

In [ ]:
# Preprocessing
# code here
def remove_all_punctuations(data):
    all_punctuations =  list(string.punctuation)
    all_punctuations.append("``")
    lst = []
    for review in data:
        tokenized = word_tokenize(review)
        considered_word = [word for word in tokenized if not word in all_punctuations]
        considered_word = " ".join(considered_word)
        lst.append(considered_word)
    return lst

def remove_hashtags(data):
        data= [review.replace("#", '"') for review in data]
        return data

# code here
def remove_stop_words(data):
    stop_words = set(stop)
    lst = []
    for review in data:
        tokenized = word_tokenize(review)
        considered_word = [word for word in tokenized if not word in stop_words]
        considered_word = " ".join(considered_word)
        lst.append(considered_word)
    return lst

def remove_links(data):
    ans = []
    for d in data:
         ans.append(re.sub(r"http\S+", "", d))
    return ans

def clean_html(data):
    ans = []
    html = re.compile('<.*?>')
    for text in data:
        
        ans.append( html.sub(r'', text) )
    return ans

def remove_non_ascii(data):
    ans =[]
    for s in data:
        ans.append("".join(i for i in s if ord(i)<128))
    return ans

def clean_links(data):
    ans = []
    txt = re.compile('http(s)?://\w+(\.\w+){1,}(/\w+)*')
    for text in data:
        ans.append(txt.sub(r'', text))
    return ans

def clean_links_a1(data):
    ans = []
    txt = re.compile(r'//t.co/[a-zA-Z0-9_]*')
    for text in data:
        ans.append(txt.sub(r'',text))
    return ans


## Reading Data & Missing Values 

In [ ]:
DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING , names=DATASET_COLUMNS)
df = dataset
df.head()

In [ ]:
X = df['text']
y = df['sentiment']

X_dev, X_test, y_dev, y_test = train_test_split(X,y,test_size=0.2)

print(X_dev.shape)
print(X_test.shape)
print(y_dev.shape)
print(y_test.shape)

## Cleaning TEXT for NLP 

In [ ]:
X_dev = remove_hashtags(X_dev)
print("Removed HashTags")
X_dev = clean_links(X_dev)
print("Removed Links")
#//t.co/DIID0JGmaX
X_dev = clean_links_a1(X_dev)
print("Removed Links Part 2")
X_dev = remove_stop_words(X_dev)
print("Removed Stopwords")
X_dev = remove_all_punctuations(X_dev)
print("Removed Punctuations")

## Lemmatizing 

In [ ]:
#code here
#Ref : From lecture notest
def stemSentence(sentence):
         porter = PorterStemmer()
         token_words = word_tokenize(sentence)
         stem_sentence = [porter.stem(word) for word in token_words]
         return " ".join(stem_sentence.copy())
print("Starting lemmatization ")

X_dev = [stemSentence(review) for review in X_dev ]
print("Xdev lemmatization done")
X_test = [stemSentence(review) for review in X_test ]
print("Xtestlemmatization done")

print("Lemmatization Completed")


## TFIDF Vectorization

In [ ]:
#code here
tfidf_vector = TfidfVectorizer()
X_dev_tfidf = tfidf_vector.fit_transform(X_dev)
print("Done vectorization for X Dev")
X_dev_tfidf = tfidf_vector.transform(X_dev)
print("Done vectorization for X test")

## Vanilla Vectorization

In [ ]:
vector = CountVectorizer()
X_dev_text = vector.fit_transform(X_dev)
print("Done vectorization for X Dev")
X_test_text = vector.transform(X_test)
print("Done vectorization for y dev")

## Fit Vanilla Model

In [ ]:
model =  LinearSVC(random_state=0, tol=1e-5, verbose=1)
model.fit(X_dev_text, y_dev)
# save the model to disk

## Fit Tfidf Model

In [ ]:
lr_tfid =  LinearSVC(random_state=0, tol=1e-5, verbose=1)
lr_tfid.fit(X_dev_tfidf,y_dev)
print("Logistic Regression Completed for TFIDF Phase")

## Saving Model Files

In [ ]:
filename = 'finalized_model_SVM.sav'
pickle.dump(model, open(filename, 'wb'))

filename_tfidf = 'finalized_model_tfidf_SVM.sav'
pickle.dump(model, open(filename_tfidf, 'wb'))

pickle.dump(vector, open("vectorizer_SVM.pickle", "wb"))

pickle.dump(tfidf_vector, open("vectorizer_tfidf_SVM.pickle", "wb"))

In [ ]:
## Using Pretrained Model
directory_path = "/kaggle/input/sample"
!zip -r file.zip directory_path

### End of file

## Transformers Model

In [15]:
df = pd.read_csv('/kaggle/input/sample/scraped_tweets.csv')

In [16]:
drop_columns = ['description', 'location','Unnamed: 0','username']
data = df.drop(columns= drop_columns)

In [84]:
X_dev = data['text']

### Convert to tokens

In [36]:
def find_sentiment(reviews):
     l = []
     for review in tqdm(reviews):
        review = review.lower()
        tokens = tokenizer.encode(review, return_tensors='pt')
        result = model(tokens)
        l.append(int(torch.argmax(result.logits))+1)
     return l

### Find results using model

In [37]:
ans = find_sentiment(X_dev)

100%|██████████| 1500/1500 [04:41<00:00,  5.33it/s]


In [40]:
res = list(zip(*np.unique(ans, return_counts=True)))

In [45]:
ans2 = find_sentiment(X_dev)

100%|██████████| 1500/1500 [04:38<00:00,  5.38it/s]


In [51]:
res2 = list(zip(*np.unique(ans2, return_counts=True)))

### Calculate output

In [77]:
X_dev = pd.DataFrame(X_dev)

In [90]:
dataframe = pd.DataFrame(X_dev.tolist(), columns=['review'])

In [93]:
dataframe['sentiment'] = ans

In [95]:
for d in datfrrame 

1    643
5    438
4    189
3    167
2     63
Name: sentiment, dtype: int64